<a href="https://colab.research.google.com/github/HansHenseler/DFRWS-APAC-LLM-Workshop/blob/main/Part_III_Hands_on_with_Llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is an example for Instruction Answering.
# You give an instruction to the LLM and the LLM makes the responses
# Model used: llama-2-13b-chat.Q4_0.gguf
# Model: Llama 2,
# Size: 13B,
# Model type: Chat model
# Quantization: 4bit, native method
# Model Format: gguf

# First step: GET A GPU !!!
# Then
# Let us install langchain and llama-cpp-python.

In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00


In [ ]:
!export LLAMA_CUBLAS=1
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 14.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.7 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.11-cp310-cp310-manylinux_2_35_x86_64.whl size=6423380 sha256=12783eb1aa2bdc47bf1706387f9f7ebecd2a226ece4e3ece47238b8e8a2da808
  Stored in directory: /root/.cache/pip/wheels/dc/42/77/a3ab0d02700427ea364de5797786c0272779dce795f62c3bc2
Successfully built llama-cpp-python


In [ ]:
# This is the chat model!
!wget https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf

--2023-10-06 15:46:49--  https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf
Resolving huggingface.co (huggingface.co)... 18.172.134.88, 18.172.134.124, 18.172.134.24, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.88|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/8d/b1/8db1d1f73b4caa58e947ccbfe2fb27ac5e495c2ad8457ad299d15987aee3b520/eda2a15d532bea4ce6fc14d15c2b72638243396816252f73a94dceeb0429112f?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-13b-chat.Q4_0.gguf%3B+filename%3D%22llama-2-13b-chat.Q4_0.gguf%22%3B&Expires=1696866409&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5Njg2NjQwOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy84ZC9iMS84ZGIxZDFmNzNiNGNhYTU4ZTk0N2NjYmZlMmZiMjdhYzVlNDk1YzJhZDg0NTdhZDI5OWQxNTk4N2FlZTNiNTIwL2VkYTJhMTVkNTMyYmVhNGNlNmZjMTRkMTVjMmI3MjYzODI0M

In [ ]:
from langchain.llms import LlamaCpp
# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="/content/llama-2-13b-chat.Q4_0.gguf", # location of the model
    temperature=0.75,                 # temperature
    max_tokens=2000,                 # Max. number of tokens to be generated
    top_p=0.9,                    # top_p = 0.9
    top_k=30,                     # top_k = 30
    n_gpu_layers=43,                 # number of layers to offload to GPU
    verbose=True, # Verbose is required to pass to the callback manager
    n_batch=200,          # number of token generation in parallel
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
!nvidia-smi

Fri Oct  6 15:49:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    28W /  70W |   7949MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
llm("what is water?")

'\n\nWater is a clear, colorless, odorless liquid that is essential for life on Earth. It is the most abundant substance on our planet, covering about 71% of the surface. Water is composed of two hydrogen atoms and one oxygen atom (H2O), and it is classified as a polar molecule due to its unequal sharing of electrons between the hydrogen and oxygen atoms.\n\nWater plays a crucial role in many biological processes, including:\n\n1. Cellular functioning: Water is essential for maintaining the proper balance of fluids and electrolytes within cells. It helps to regulate body temperature, transports nutrients and waste products, and supports metabolic activities.\n2. Digestion: Water is involved in the breakdown and absorption of nutrients from food. It helps to liquefy food in the stomach and small intestine, allowing nutrients to be absorbed into the bloodstream.\n3. Blood circulation: Water helps to maintain blood pressure and supports the proper functioning of the cardiovascular system.

In [ ]:
llm("plan a 2 days trip in Singapore")

Llama.generate: prefix-match hit


"\n\nI'm planning to visit Singapore for 2 days, I would like some suggestions on what I can do and see during this time. I would be arriving at Changi airport in the morning and leaving in the evening.\n\nI would appreciate any recommendations on the following:\n\n* Accommodation (preferably within walking distance from a MRT station)\n* Must-see attractions\n* Food places to try\n* Public transportation options\n* Safety tips\n\nPlease let me know if you have any specific questions or requirements for your recommendations. Thank you!"

In [ ]:
txt = '''
1. Gardens by the Bay - I want to see the Supertree Grove and the Flower Dome and Cloud Forest Domes.
2. Marina Bay Sands - I want to take a photo with the iconic hotel and enjoy the light and water show at night.
3. Merlion - I want to see the famous statue and take a photo with it.
4. Singapore Flyer - I want to ride the giant Ferris wheel and enjoy the panoramic view of the city.
5. Chinatown - I would like to explore the colorful streets and try some delicious street food.
6. Little India - I would like to experience the vibrant Indian culture and try some authentic Indian cuisine.
7. Haw Par Villa - I want to see the largest theme park in Singapore and enjoy the Chinese mythology-themed rides and attractions.
8. Clarke Quay - I would like to visit the nightlife district and enjoy the bars, clubs, and live music performances.
'''
llm("Please summarize the following passage in 50 words. Passage: " + txt)

Llama.generate: prefix-match hit


'\nHere is a summary of the passage in 50 words:\n\nI want to visit Gardens by the Bay, Marina Bay Sands, Merlion, Singapore Flyer, Chinatown, Little India, Haw Par Villa, and Clarke Quay to experience the vibrant culture and attractions that Singapore has to offer.'

In [ ]:
llm("which countries are located in Europe")

ValueError: ignored